# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): (впишите свои фамилию и имя)

### Общая информация

__Дата выдачи:__ 06.04.2020

__Дедлайн:__ 23:59 20.04.2020


### Оценивание и штрафы

Оценка за ДЗ вычисляется по следующей формуле:

$$
min(\text{points}, 18)  \times 10 / 18,
$$

где points — количество баллов за обязательную часть, которое вы набрали. Максимальное число баллов, которое можно получить за обязательную часть — 18, за каждые полтора балла сверху вы получите 1 бонусный балл (максимум 2). Также вы можете использовать бонусные баллы, которые накопили ранее.

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи

Загрузка файлов с решениями происходит в системе [Anytask](https://anytask.org/).

Инвайт для группы ИАД-6: rd5CNrr

Перед отправкой перезагрузите ноутбук и проверьте, что все ячейки могут быть последовательно выполнены. Ноутбук должен запускаться с использованием python 3.6+

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [39]:
# Качаем датасет

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

--2020-04-18 18:07:23--  https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tg55q9mrziroyrs/train_subset.csv [following]
--2020-04-18 18:07:23--  https://www.dropbox.com/s/raw/tg55q9mrziroyrs/train_subset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc84aa660aa16a6b7d1b3fd809d5.dl.dropboxusercontent.com/cd/0/inline/A2HoYxvddEM1q5qsrgySwZYHKBNfZu7KbU7kfLgHsMQfJhSP-TJtBLYJVQ3hCyYsh4Zq5CKVKWWoT0Tir_tjxvL28LIXGyJWLbis5La6LH4adfXjtkhbO-KqxK-yAlhyfAY/file# [following]
--2020-04-18 18:07:24--  https://uc84aa660aa16a6b7d1b3fd809d5.dl.dropboxusercontent.com/cd/0/inline/A2HoYxvddEM1q5qsrgySwZYHKBNfZu7KbU7kfLgHsMQfJhSP-TJtBLYJVQ3hCyYsh4Zq5CKVKWWoT0Tir_tjxvL28LIXGyJWLbis5La6LH4adfXjtkhbO-KqxK-yAlh

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [40]:
data = pd.read_csv("train_subset.csv.1", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [41]:
data.shape

(30000, 4)

In [42]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [316]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [345]:
X_test[:3]

array([['винт сдвижной панели потолка бмв х5 е53',
        '"* 3308 * винт сдвижной панели потолка oem : 54107199476 ( 54 10 7 199 476 ) / / запчасть б / у в хорошем состоянии для bmw x5 e53 , / демонтирована с автомобиля из японии без пробега по рф . гарантия на все запчасти 14 дней . возможна замена / установка в автосервисе партнеров . / примечания : / / для упрощения поиска запчасти сообщите нам внутренний код : * 3308 * / в наличии на складе : 16 шт . / проверяйте соответствие данной запчасти по оригинальному номеру оем - вашему автомобилю по vin . более 1000 наименований запасных частей для bmw на нашем складе в москве . отправляем по всей россии почтой или транспортными компаниями "'],
       ['сандалики totto для мальчика',
        'продаю сандалики totto для малыша ! размер 20 , по стельке 12 см .'],
       ['фара правая toyota rav 4 галоген 2015 - 19',
        'фара правая для toyota rav4 2015 / оригинальный номер : 8113042650 / тойота рав4 тоета рав 4 / производитель : toyot

In [294]:
y_train[:5]

array([ 27,  20,  84, 106,  27])

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Можете использовать WordPunctTokenizer или подобрать какой-то другой, если считаете, что он лучше подойдет для этой задачи.


In [351]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", tokenizer.tokenize(text),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: ['Здраствуйте', '.', 'Я', ',', 'Кирилл', '.', 'Хотел', 'бы', 'чтобы', 'вы', 'сделали', 'игру', ',', '3Д', '-', 'экшон', 'суть', 'такова', '...']


__Задание:__ реализуйте функцию ниже

In [347]:
def preprocess(text: str) -> str:
    """
    Данная функция принимает на вход текст, 
    а возвращает тот же текст, но с пробелами между каждым токеном
    """
    
    return ' '.join(tokenizer.tokenize(text.lower()))

In [348]:
assert preprocess(text) == 'здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...'

__Задание:__ токенизируйте title и description в train и test

In [318]:
for i in range(len(X_train)):
    for j in range(2):
        X_train[i][j] = preprocess(X_train[i][j])    

In [319]:
for i in range(len(X_test)):
    for j in range(2):
        X_test[i][j] = preprocess(X_test[i][j])

In [320]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

In [349]:
X_train[:3]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .']],
      dtype=object)

### BOW (3 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ создайте словарь, где в соответствии каждому токену стоит количество раз, которое оно встретилось в X_train

In [161]:
tokens =  dict()
for i in range(len(X_train)):
    for j in  range(2):
        string = X_train[i][j].split()
        for word in string:
            if word not in tokens:
                tokens[word] = 1
            else:
                tokens[word] += 1

In [162]:
assert tokens['сапоги'] == 454

In [236]:
len(tokens), tokens

(94292,
 {'сапоги': 454,
  '46': 471,
  'размер': 3320,
  'новые': 1872,
  'светильники': 26,
  'потолочный': 8,
  'swarovski': 9,
  'потолочные': 8,
  '6': 2126,
  'штук': 135,
  ',': 79117,
  'цена': 2379,
  'за': 3425,
  'штуку': 134,
  '.': 65624,
  'в': 28337,
  'эксплуатации': 157,
  '2': 5766,
  'года': 1012,
  'продаются': 152,
  'связи': 322,
  'со': 853,
  'сменой': 10,
  'интерьера': 39,
  'квартире': 261,
  'iphone': 465,
  '7': 1485,
  'plus': 163,
  '128gb': 25,
  'red': 34,
  'красный': 121,
  'наличии': 2172,
  '/': 85802,
  'данная': 149,
  'только': 1129,
  'для': 9627,
  'подписчиков': 2,
  'instagram': 39,
  ':': 15098,
  'iqmac': 1,
  'новый': 1626,
  'айфон': 37,
  'это': 744,
  'элегантный': 14,
  'и': 21714,
  'мощный': 53,
  'смартфон': 70,
  'который': 200,
  'готов': 82,
  'полной': 71,
  'мере': 9,
  'раскрыть': 4,
  'возможности': 70,
  'ios': 37,
  '10': 2452,
  'аппарат': 117,
  'с': 12860,
  '4': 2939,
  '-': 36840,
  'ядерным': 1,
  'процессором': 5,
  

__Задание:__ выведите 10 самых частотных и 10 самых редких токенов

In [163]:
ten_most_common = sorted(tokens, key=tokens.get, reverse=True)[:10]
ten_least_common = sorted(tokens, key=tokens.get, reverse=False)[:10]

__Задание:__ оставьте в словаре только топ 10000 самых частотных токенов

In [164]:
most_common = sorted(tokens, key=tokens.get, reverse=True)[:10000]

__Задание:__ реализуйте функцию, которая предложение переводит в вектор из чисел. То есть каждому слову из словаря сопоставляется количество раз, которое оно встретилось в предложении.

In [165]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    input: строка
    output: вектор размерности словаря
    """
    vector = []
    text = preprocess(text)
    text = text.split()
    for word in most_common:
        vector.append(text.count(word))
    return np.array(vector)

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и для каждого текста из description сопоставляет вектор.

In [166]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    
    matrix = []
    for i in range(len(items)):
        matrix.append(text_to_bow(items[i][1]))
    return np.array(matrix)
    # your code here

In [179]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

In [212]:
X_train_bow[:2], X_test_bow[:2], y_train[:2], y_test[:2]

(array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.38490018, 0.19245009, ..., 0.        , 0.        ,
         0.        ]]),
 array([[0.57353933, 0.05735393, 0.3441236 , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.2236068 , 0.2236068 , ..., 0.        , 0.        ,
         0.        ]]),
 array([27, 20]),
 array([10, 29]))

In [72]:
# чтобы видеть проход по итерациям можно использовать библиотеку tqdm
# она работает примерно так
from tqdm import tqdm

for i in tqdm(range(100)):
    pass

100%|██████████| 100/100 [00:00<00:00, 14340.48it/s]


In [152]:
from sklearn.metrics import accuracy_score

### Логистическая регрессия и SVC (0.5 балла)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


In [190]:
from scipy.sparse import csr_matrix
xTrain_bow = csr_matrix(X_train_bow)
xTest_bow = csr_matrix(X_test_bow)

In [211]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver='lbfgs',  max_iter=1000)
lr_model = lr_model.fit(xTrain_bow, y_train)

accuracy_score(lr_model.predict(xTest_bow), y_test)

0.6533333333333333

In [ ]:
assert accuracy_score(lr_model.predict(xTest_bow), y_test) > 0.7

In [173]:
from sklearn.svm import LinearSVC

svc_model = LinearSVC(max_iter=1000)
svc_model.fit(xTrain_bow, y_train)

accuracy_score(svc_model.predict(xTest_bow), y_test)

0.684

In [174]:
assert accuracy_score(svc_model.predict(xTest_bow), y_test) > 0.68

### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?

In [177]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    
    matrix = []
    for i in range(len(items)):
        matrix.append(text_to_bow(items[i][1] + items[i][0]))
    return np.array(matrix)

Нормализуйте данные (`sklearn.preprocessing.normalize`) перед обучением. Что станет с качеством и почему?

In [213]:
from sklearn.preprocessing import normalize
X_train_bow_norm = sklearn.preprocessing.normalize(X_train_bow, axis=1)
X_test_bow_norm = sklearn.preprocessing.normalize(X_test_bow, axis=1)

In [216]:
X_train_bow_norm[:2], X_test_bow_norm[:2], y_train_norm[:2], y_test_norm[:2]

(array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.38490018, 0.19245009, ..., 0.        , 0.        ,
         0.        ]]),
 array([[0.57353933, 0.05735393, 0.3441236 , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.2236068 , 0.2236068 , ..., 0.        , 0.        ,
         0.        ]]),
 array([0.00399414, 0.00295862]),
 array([0.00221741, 0.00643049]))

In [218]:
from scipy.sparse import csr_matrix
xTrain_bow_norm = csr_matrix(X_train_bow_norm)
xTest_bow_norm = csr_matrix(X_test_bow_norm)

In [417]:
from sklearn.svm import LinearSVC

svc_model = LinearSVC(max_iter=1000)
svc_model.fit(xTrain_bow_norm, y_train)

accuracy_score(svc_model.predict(xTest_bow_norm), y_test)

0.7745555555555556

### Mystem (1 балл)

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [390]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

--2020-04-20 11:18:55--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.243, 5.45.205.244, 5.45.205.245, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.243|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskmar09.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-04-20 11:18:55--  http://cache-mskmar09.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskmar09.cdn.yandex.net (cache-mskmar09.cdn.yandex.net)... 5.45.222.33
Connecting to cache-mskmar09.cdn.yandex.net (cache-mskmar09.cdn.yandex.net)|5.45.222.33|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.2’

mystem-3.0-linux3.1 100%[===================>]  15.70M  7.28MB

In [391]:
pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to /private/var/folders/7z/tfzx8f8j65j45v0xnyhz3b840000gn/T/pip-req-build-hkwcdber
  Running command git clone -q https://github.com/nlpub/pymystem3 /private/var/folders/7z/tfzx8f8j65j45v0xnyhz3b840000gn/T/pip-req-build-hkwcdber
  Created wheel for pymystem3: filename=pymystem3-0.2.0-py3-none-any.whl size=9920 sha256=dd2cf165fa42274237f840e15e695c631c5e6ff6840b0e606c5bdd6d6641c8da
  Stored in directory: /private/var/folders/7z/tfzx8f8j65j45v0xnyhz3b840000gn/T/pip-ephem-wheel-cache-gjny9s31/wheels/72/92/6b/9521f1a0b0f77ec13f07bf5f87c50c531b1d8686cf8ad48b43
Successfully built pymystem3
Note: you may need to restart the kernel to use updated packages.


In [398]:
from pymystem3 import Mystem

# Your code here
m = Mystem()

Installing mystem to /Users/kirilldokj/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


In [400]:
def  text_mystem(text):
    for i in range(len(text)):
        for j in range(2):
            text[i][j] = ' '.join(m.lemmatize(text[i][j]))
    return text

In [404]:
m_X_train = text_mystem(X_train)
m_X_test = text_mystem(X_test)

In [405]:
m_X_train[:3]

array([['сапог       46       размер       новый   \n',
        'сапог       46       размер       новый   \n'],
       ['светильник       потолочный       swarovski   \n',
        'светильник       потолочный       swarovski       6       штука    ,    цена       за       штука       .       в       эксплуатация       2       год    ,    продаваться       в       связь       со       смена       интерьер       в       квартира   \n'],
       ['iphone       7       plus       128gb       red       красный       в       наличие   \n',
        '/ /    данный       цена       только       для       подписчик       instagram    :    iqmac    / /    новый       красный       айфон       7       plus       в       наличие       это       элегантный       и       мощный       смартфон    ,    который       готовый       в       полный       мера       раскрывать       новый       возможность       ios       10       .       аппарат       с       4    -    ядерный       процессор       а10    

In [408]:
tokens_m =  dict()
for i in range(len(m_X_train)):
    for j in  range(2):
        string = m_X_train[i][j].split()
        for word in string:
            if word not in tokens_m:
                tokens_m[word] = 1
            else:
                tokens_m[word] += 1

In [410]:
m_most_common = sorted(tokens_m, key=tokens_m.get, reverse=True)[:10000]

In [411]:
def m_text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    input: строка
    output: вектор размерности словаря
    """
    vector = []
    text = preprocess(text)
    text = text.split()
    for word in m_most_common:
        vector.append(text.count(word))
    return np.array(vector)

In [412]:
def m_items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    
    matrix = []
    for i in range(len(items)):
        matrix.append(m_text_to_bow(items[i][1] + items[i][0]))
    return np.array(matrix)

In [413]:
m_X_train_bow = items_to_bow(m_X_train)
m_X_test_bow = items_to_bow(m_X_test)

In [414]:
m_xTrain_bow = csr_matrix(m_X_train_bow)
m_xTest_bow = csr_matrix(m_X_test_bow)

In [415]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver='lbfgs',  max_iter=1000)
lr_model = lr_model.fit(m_xTrain_bow, y_train)

accuracy_score(lr_model.predict(m_xTest_bow), y_test)

0.77

### TF-IDF (5 баллов)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя. Для простоты посчитайте общий tf-idf для title и description (то есть каждому объекту надо сопоставить вектор, где как документ будет рассматриваться конкатенация title и description).

__Задание:__ составьте словарь, где каждому слову из изначального словаря будет стоять в соответствии количество документов, где это слово встретилось.

In [375]:
X_train[:3]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .']],
      dtype=object)

In [244]:
newDict = {}
for i in range(len(X_train)):
    string = set(X_train[i][0].split() + X_train[i][1].split())
    for word in string:
        if word not in newDict:
            newDict[word] = 1
        else:
            newDict[word] += 1                                      

In [245]:
len(newDict),  newDict

(94292,
 {'сапоги': 285,
  '46': 391,
  'размер': 2839,
  'новые': 1519,
  'штук': 112,
  'потолочный': 6,
  'сменой': 10,
  'цена': 2041,
  'штуку': 125,
  'потолочные': 8,
  'в': 11893,
  'эксплуатации': 147,
  ',': 14630,
  'со': 782,
  'светильники': 20,
  '2': 3529,
  '.': 15735,
  '6': 1548,
  'за': 2802,
  'связи': 308,
  'продаются': 146,
  'swarovski': 7,
  'года': 882,
  'интерьера': 38,
  'квартире': 210,
  '4': 2054,
  'приложений': 4,
  'мере': 8,
  'задачи': 22,
  'iqmac': 1,
  'элегантный': 13,
  '5': 2731,
  'а10': 2,
  'процессором': 5,
  'это': 607,
  '«': 656,
  'и': 8355,
  'красный': 106,
  'с': 6746,
  '7': 1155,
  'наличии': 1814,
  'для': 5534,
  '3': 2711,
  'red': 27,
  'ipad': 24,
  'экран': 131,
  'решает': 4,
  'только': 986,
  'быстродействием': 1,
  'готов': 79,
  'ресурсоемкие': 1,
  'дисплее': 8,
  'аппарат': 56,
  '10': 1827,
  'pro': 102,
  'что': 830,
  'озу': 22,
  'который': 182,
  'игр': 82,
  'iphone': 221,
  'так': 1212,
  'ядерным': 1,
  'тепер

In [237]:
newDict_common = sorted(tokens, key=tokens.get, reverse=True)[:10000]


__Задание:__ реализуйте функцию, где тексту в соответствие ставится tf-idf вектор. 

In [253]:
def text_to_tfidf(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из словаря
    указан tf-idf
    """
    vector = []
    text = preprocess(text).split()
    for word in newDict_common:
        if word in text:
            tf = text.count(word) / len(text)
            idf = np.log(len(X_train) / newDict[word])
            vector.append(tf*idf)
        else:
            vector.append(0)
    return np.array(vector)

In [254]:
example_1 = 'за связи приложение смартфон раскрыть тяжелый'
text_to_tfidf(example_1)

array([0., 0., 0., ..., 0., 0., 0.])

In [255]:
example_2 = 'сапоги сапоги новые сапоги потолочный потолочные в'
text_to_tfidf(example_2)
np.sum(text_to_tfidf(example_2))

2.29920118134269

In [262]:
example_3 = X_train[2][1] + X_train[2][0]
text_to_tfidf(example_3)

array([0.03384833, 0.01882543, 0.00901984, ..., 0.        , 0.        ,
       0.        ])

In [263]:
len(text_to_tfidf(example_3))

10000

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и для каждого объекта сопоставляет вектор tf-idf.

In [257]:
def items_to_tfidf(items: np.array) -> np.array:
    """ 
    Для каждого товара возвращает его tfidf вектор
    """
    matrix = []
    for i in range(len(items)):
        matrix.append(text_to_bow(items[i][1] + items[i][0]))
    return np.array(matrix)

In [258]:
X_train[2][1] + X_train[2][0]

'/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .iphone 7 plus 128gb red красный в наличии'

In [261]:
example_4 = preprocess(X_train[2][1] + X_train[2][0]).split()
example_4[:12]

['/',
 '/',
 'данная',
 'цена',
 'только',
 'для',
 'подписчиков',
 'instagram',
 ':',
 'iqmac',
 '/',
 '/']

In [264]:
X_train_tfidf = items_to_tfidf(X_train)
X_test_tfidf = items_to_tfidf(X_test)

In [265]:
X_train_tfidf, X_test_tfidf

(array([[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  2,  1, ...,  0,  0,  0],
        [ 4,  5,  3, ...,  0,  0,  0],
        ...,
        [42, 15,  4, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 1,  0,  4, ...,  0,  0,  0]]),
 array([[10,  1,  6, ...,  0,  0,  0],
        [ 0,  1,  1, ...,  0,  0,  0],
        [10,  1,  2, ...,  0,  0,  0],
        ...,
        [10,  1,  3, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0, 10,  9, ...,  0,  0,  0]]))

In [266]:
xTrain_tfidf = csr_matrix(X_train_tfidf)
xTest_tfidf = csr_matrix(X_test_tfidf)

In [268]:
xTrain_tfidf

<21000x10000 sparse matrix of type '<class 'numpy.longlong'>'
	with 733986 stored elements in Compressed Sparse Row format>

__Задание:__ обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [269]:
from sklearn.linear_model import LogisticRegression


lr_model = LogisticRegression(solver='lbfgs',  max_iter=10000)
lr_model = lr_model.fit(xTrain_tfidf, y_train)

accuracy_score(lr_model.predict(xTest_tfidf), y_test)

0.75

In [273]:
from sklearn.svm import LinearSVC

svc_model = LinearSVC(max_iter=1000) 
svc_model.fit(xTrain_tfidf, y_train)

accuracy_score(svc_model.predict(xTest_tfidf), y_test)

0.7302222222222222

### Hashing Vectorizer (1 балл)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [331]:
X_train[:3]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .']],
      dtype=object)

In [377]:
def title_descrip(text):
    """Функция объеденияет title и description"""
    new_text = []
    for i in range(len(text)):
        new_text.append([" ".join(text[i][0].split() + text[i][1].split())])
    return new_text

In [386]:
X_train_new = title_descrip(X_train)
X_test_new = title_descrip(X_test)
X_train_new[3]

['пион ирис ромашка рассада пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п . зубчаниновка / либо пл . революции / есть ирисы , ромашка , клубника , боярышник и ирга']

In [387]:
for i in range(len(X_train_new)):
    X_train_new[i][0] = X_train_new[i][0].split()   

In [388]:
X_train_new[4][0]

['кофта', 'состояние', 'отличное']

In [393]:
from sklearn.feature_extraction.text import HashingVectorizer

### Word Vectors (3 балла)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах изинтернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [422]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-04-20 12:36:58--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-04-20 12:36:59--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2dc4a7a4814f0c0773041b7f8b.dl.dropboxusercontent.com/cd/0/inline/A2MZa-x5d1vBPw_4HP2Cog6SPGDywGRufpSjkEuQh95Q5a5p5POlY2Wd0R6n5RNd7d1Dtim6twAbuC7Ncpjpef1fNZjRPGBHXgWhphQXML_TIQ/file# [following]
--2020-04-20 12:37:00--  https://uc2dc4a7a4814f0c0773041b7f8b.dl.dropboxusercontent.com/cd/0/inline/A2MZa-x5d1vBPw_4HP2Cog6SPGDywGRufpSjkEuQh95Q5a5p5POlY2Wd0R6n5RNd7d1Dtim6twAbuC7Ncpjpef1fNZjRPGBHXgWhphQXML_TIQ/file
Resolving uc2dc4a7a4814f0c0773041b7f8b.dl.dropboxuser

In [425]:
!tar -xzf ru.tar.gz

In [420]:
!pip install gensim

     |████████████████████████████████| 23.7 MB 8.0 MB/s eta 0:00:01    |███████                         | 5.2 MB 3.0 MB/s eta 0:00:07     |███████▌                        | 5.6 MB 3.0 MB/s eta 0:00:06     |████████▊                       | 6.5 MB 3.0 MB/s eta 0:00:06
     |████████████████████████████████| 105 kB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 6.0 MB/s eta 0:00:011
     |████████████████████████████████| 6.1 MB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 7.7 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.11.1-py3-none-any.whl size=95255 sha256=b297c34905d70e7349f18a332776aa8143b770bd138ab0dbd64d82637146c316
  Stored in directory: /Users/kirilldokj/Library/Caches/pip/wheels/1a/8c/a2/7b24df77c58dab0aec275c1bd3d4392d54df941020199f6759
Successfully built smart-open
  Attempting uninstall: docutils
    Found existing installation: docutils

In [421]:
import gensim
from gensim.models.wrappers import FastText

model = FastText.load_fasttext_format('ru.bin')

unable to import 'smart_open.gcs', disabling that module


In [466]:
# как мы видим, каждому слову данная модель сопоставляет вектор размерности 300

print(model['привет'].shape)
print(model['привет'][::20])

(300,)
[ 0.02916384  0.11941359 -0.0171444  -0.05672329 -0.01074128  0.02357679
  0.00284878  0.05847022  0.08947854 -0.05106218 -0.00062491 -0.03419575
 -0.04124978 -0.04706197  0.07113555]


In [436]:
import re 

def clean_sentence(text):
    text = re.sub(r'\W',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text
s = 'привет, пока, а вот, тут ,и. я!'
clean_sentence(s)

'привет пока а вот тут и я '

In [442]:
# Будем рассматривать эмбеддинг предложения, как сумму эмбеддингов токенов

def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """
    sentence = clean_sentence(sentence).split()
    vector = np.zeros(300, )
    for word in sentence:
        if  word in  model:
            vector += model[word]
    return vector 

In [448]:
sent = 'сдаётся уютный , тёплый гараж для стартапов в ml'
sentence_embedding(sent)[::50]

array([ 0.08189847,  0.07249197, -0.15601223,  0.03782297,  0.09215296,
       -0.23092947])

In [446]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

__Задание:__ сделайте все то же, что в предыдущих пунктах -- реализуйте функцию, которая преобразует данные, а затем обучите логистическую регрессию и SVM, оцените качество.

In [454]:
def item_embedding(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    
    matrix = []
    for i in range(len(items)):
        matrix.append(sentence_embedding(items[i][1] + items[i][0]))
    return np.array(matrix)

In [457]:
embed_X_train = item_embedding(X_train)
embed_X_test = item_embedding(X_test)

In [456]:
embed_X_train

array([[-5.36972042e-02, -1.96721570e-01,  7.15154782e-02, ...,
        -4.59109694e-02,  2.81868301e-01, -2.47110227e-01],
       [ 1.76217829e-01, -4.93197911e-02,  1.63144056e-01, ...,
         2.79929767e-01, -1.06307602e-01,  5.52765599e-02],
       [ 3.73223361e-01,  3.82948786e-01,  2.30265993e-01, ...,
         9.25260585e-02,  4.14456744e-01, -1.70334767e+00],
       ...,
       [ 4.20926411e+00,  3.41696435e-02,  2.15725750e+00, ...,
         1.74950989e+00, -4.33913203e+00, -1.80054314e+00],
       [ 3.53956436e-03, -1.43398473e-01, -3.47132912e-02, ...,
         9.67901660e-02, -1.79040452e-01, -6.68929252e-02],
       [-2.28068327e-01, -1.66307816e-01,  1.35653891e-01, ...,
         8.60630419e-02,  4.40917619e-01,  3.21026361e-01]])

In [458]:
xTrain_embed = csr_matrix(embed_X_train)
xTest_embed = csr_matrix(embed_X_test)

In [459]:
lr_model = LogisticRegression(solver='lbfgs',  max_iter=10000)
lr_model = lr_model.fit(xTrain_tfidf, y_train)

accuracy_score(lr_model.predict(xTest_tfidf), y_test)

0.75

In [460]:
svc_model = LinearSVC(max_iter=1000)
svc_model.fit(xTrain_bow, y_train)

accuracy_score(svc_model.predict(xTest_bow), y_test)

0.7745555555555556

### Что дальше? (6 баллов)

Для получения максимальной оценки вам нужно решить любые 2 пункта. Решение каждого пункта даст вам полтора балла:

1. Реализовать N-Gram модели текстовой классификации (__1.5 балла__)

2. Поработать с другими эмбеддингами для слов (например word2vec или GloVe) (__1.5 балла__)

3. Другие способы токенизации (pymorphy2, spaCy) (__1.5 балла__)

4. Добиться качества > 0.765 на тестовых данных (попробуйте другие токенизаторы, предобработку текста, и любые другие идеи, которые вам придут в голову) (__1.5 балла__)

Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

In [ ]:
# Добьемся  качества > 0.765 

In [365]:
import re 


def clean_text(text):
    for i in range(len(text)):
        ignore_words = ['а', 'с', 'под', 'на', 'над', 'и', 'к', 'в', 'из', 'по']
        text[i][0] = text[i][0].lower()
        text[i][0] = re.sub(r'\W',' ',text[i][0])
        text[i][0] = re.sub(r'\s+',' ',text[i][0])
        text[i] = ' '.join([w for w in  text[i][0].split() if w not in ignore_words])
    return text

In [366]:
X_train_2 = title_descrip(X_train)
X_test_2 =  title_descrip(X_test)

In [367]:
X_train_2_cleaned = clean_text(X_train_2)
X_test_2_cleaned = clean_text(X_test_2)

In [368]:
X_train_2_cleaned[:4]

['сапоги 46 размер новые сапоги 46 размер новые',
 'светильники потолочный swarovski светильники потолочные swarovski 6 штук цена за штуку эксплуатации 2 года продаются связи со сменой интерьера квартире',
 'iphone 7 plus 128gb red красный наличии данная цена только для подписчиков instagram iqmac новый красный айфон 7 plus наличии это элегантный мощный смартфон который готов полной мере раскрыть новые возможности ios 10 аппарат 4 ядерным процессором а10 3 гб озу легкостью решает самые ресурсоемкие задачи позволяя наслаждаться быстродействием тяжелых приложений игр 5 5 дюймовом дисплее аппарат получил экран как у ipad pro так что картинка теперь соответствует кинематографическому стандарту',
 'пион ирис ромашка рассада пион куст 500 р более 10 шт саженец корень 100р растут у нас более 70 лет розовые бордовые белые фото цветы 2018г п зубчаниновка либо пл революции есть ирисы ромашка клубника боярышник ирга']

In [369]:
def items_to_bow_new(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    
    matrix = []
    for i in range(len(items)):
        matrix.append(text_to_bow(items[i]))
    return np.array(matrix)

In [371]:
X_train_bow_new = items_to_bow_new(X_train_2_cleaned)
X_test_bow_new = items_to_bow_new(X_test_2_cleaned)

In [372]:
X_train_bow_new[:2], X_test_bow_new[:2]

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [373]:
from scipy.sparse import csr_matrix
xTrain_bow_new = csr_matrix(X_train_bow_new)
xTest_bow_new = csr_matrix(X_test_bow_new)

In [374]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver='lbfgs',  max_iter=1000)
lr_model = lr_model.fit(xTrain_bow_new, y_train)

accuracy_score(lr_model.predict(xTest_bow_new), y_test)

0.7826666666666666